<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [プリント基板の電子部品検出 Nishika株式会社](https://competition.nishika.com/competitions/kiban/summary)

## Build Environment

/content/drive/MyDrive/PCB_Detect/: Working directory  
┣ 01_input: A directory includes input data in competition  
┃ ┣ train_imgs.zip: A zip file including train images    
┃ ┣ test_imgs.zip: A zip file including test images  
┃ ┣ train.csv:  A csv file in which table data written for training.   
┃ ┣ test.csv: A csv file in which table data written for test.  
┃ ┗ sample_submission.csv: A csv file is sample for submission.  
┣ 03_code: A directory for main code.
┣ 04_model: A directory for model.  
┗ 05_submission: A directory for submission data file.  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
TEMP_DIR = "./temp"
INPUT_DIR:str = "/content/drive/MyDrive/PCB_Detect/01_input"
#TRAIN_DIR:str = INPUT_DIR + "/train_data"
#TEST_DIR:str = INPUT_DIR + "/test_data"
#ANALYSIS_DIR:str = "/content/drive/MyDrive/Human_Activity_Recognition_with_Smartphones/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/PCB_Detect//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/PCB_Detect/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

In [3]:
import os, shutil, glob, zipfile

if not os.path.isdir('./train_imgs') and  not os.path.isdir('./test_imgs'):
  # Copy ZIP files including images, and Unzip them.
  !cp -r "/content/drive/MyDrive/PCB_Detect/01_input/train_imgs.zip" ./
  !cp -r "/content/drive/MyDrive/PCB_Detect/01_input/test_imgs.zip" ./
  shutil.unpack_archive('train_imgs.zip', './')
  shutil.unpack_archive('test_imgs.zip', './')

## Define Rle decoding with tensorflow

In [4]:
import tensorflow as tf

def rle_decode_tf(mask_rle, shape):
    shape = tf.convert_to_tensor(shape, tf.int64)
    size = tf.math.reduce_prod(shape)
    # Split string
    s = tf.strings.split(mask_rle)
    s = tf.strings.to_number(s, tf.int64)
    # Get starts and lengths
    starts = s[::2] - 1
    lens = s[1::2]
    # Make ones to be scattered
    total_ones = tf.reduce_sum(lens)
    ones = tf.ones([total_ones], tf.uint8)
    # Make scattering indices
    r = tf.range(total_ones)
    lens_cum = tf.math.cumsum(lens)
    s = tf.searchsorted(lens_cum, r, 'right')
    idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
    # Scatter ones into flattened mask
    mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
    # Reshape into mask
    return tf.reshape(mask_flat, shape)

## Confirm Data

In [9]:
import pandas as pd
train = pd.read_csv(INPUT_DIR + "/train.csv")
train

,Instance ID,image,filename_index,height,width,Vertices,rle
0,0,pcb_32b_cc_12.png,pcb_32b,5474,6783,"[[[698, 4511], [698, 4436], [700, 4434], [702,...",30062961 47 30069742 50 30076523 53 30083305 5...
1,1,pcb_32b_cc_12.png,pcb_32b,5474,6783,"[[[241, 4507], [240, 4508], [240, 4535], [239,...",30571223 31 30578005 32 30584788 32 30591571 3...
2,2,pcb_32b_cc_12.png,pcb_32b,5474,6783,"[[[545, 3351], [547, 3351], [548, 3349], [548,...",22506489 15 22513271 42 22520054 55 22526837 5...
3,3,pcb_32b_cc_12.png,pcb_32b,5474,6783,"[[[281, 3410], [329, 3409], [331, 3408], [332,...",22913254 52 22920035 55 22926817 57 22933600 5...
4,4,pcb_32b_cc_12.png,pcb_32b,5474,6783,"[[[392, 3562], [384, 3563], [383, 3562], [383,...",23415311 59 23422094 60 23428877 60 23435660 6...
...,...,...,...,...,...,...,...
25698,859,pcb_51b_cc_31.png,pcb_51b,5504,8256,"[[[3824, 1798], [3823, 1799], [3821, 1799], [3...",14848113 12 14856366 16 14864622 16 14872878 1...
25699,860,pcb_51b_cc_31.png,pcb_51b,5504,8256,"[[[3730, 1799], [3738, 1799], [3740, 1799], [3...",14856274 12 14864528 15 14872784 15 14881040 1...
25700,861,pcb_51b_cc_31.png,pcb_51b,5504,8256,"[[[5739, 353], [5740, 353], [5749, 353], [5751...",2920108 13 2928364 14 2936619 15 2944875 15 29...
25701,862,pcb_51b_cc_31.png,pcb_51b,5504,8256,"[[[3640, 2536], [3640, 2553], [3649, 2553], [3...",20940857 10 20949113 10 20957369 10 20965625 1...


In [15]:
height:int = train['height'][0]
width:int = train['width'][0]
rle = train['rle'][0]
shape = [height, width]

rle_decode_tf(rle,shape)

<tf.Tensor: shape=(5474, 6783), dtype=uint8, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)>

In [21]:
import numpy as np
def parse_img(file_path,new_size=[128,128]):

    # Image
    img_content = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img_content)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img,new_size)

    # Mask
    file_id = tf.strings.split(file_path,'/')[-1]
    objects = [rle_decode(m) for m in df2[train['filename_index']==file_id]]
    mask = np.sum(objects,axis=0)
    mask = np.expand_dims(mask,3)   # Force mask to have 3 channels, necessary for resize step
    mask = tf.image.convert_image_dtype(mask, tf.int8)
    mask = tf.clip_by_value(mask,0,1)
    mask = tf.image.resize(mask,new_size)
    mask = tf.squeeze(mask)     # squeeze back
    mask = tf.image.convert_image_dtype(mask, tf.int8)

    return img, mask

In [22]:
parse_img('./train_imgs/'+train['image'][0])

NameError: name 'df2' is not defined

In [16]:
list_ds = tf.data.Dataset.list_files(train_paths_abs)
ds = list_ds.map(parse_img)

NameError: name 'train_paths_abs' is not defined